In [1]:
import os
import ccxt
import datetime
from configparser import ConfigParser
from binance.client import Client
from utils.functions_bollinger import *

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 30

RSI_PERIOD = 14
RSI_OVERBOUGHT = 55
RSI_OVERSOLD = 45

CCXT_TICKER_NAME = 'LUNA/USDT'
TRADING_TICKER_NAME = 'LUNAUSDT'

INVESTMENT_AMOUNT_DOLLARS = 10
holding_quantity = 0

exchange = ccxt.binance()
binance_client = Client(api_key=api_key, api_secret=api_secret)
DECIMALS_QUANTITY = check_decimals(TRADING_TICKER_NAME, binance_client)

## Observations:
* It is recommend to initialize the bot close to the moment that a candle is closed

## Getting the data

In [4]:
df = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
df

,at,open,high,low,close,vol,Date,symbol
0,1651080600000,89.05,89.12,88.68,89.00,33017.29,2022-04-27 17:30:00,LUNA/USDT
1,1651082400000,89.01,89.41,88.94,89.11,31082.02,2022-04-27 18:00:00,LUNA/USDT
2,1651084200000,89.11,89.32,88.64,88.72,38680.61,2022-04-27 18:30:00,LUNA/USDT
3,1651086000000,88.67,88.78,88.03,88.41,47861.70,2022-04-27 19:00:00,LUNA/USDT
4,1651087800000,88.40,88.60,88.13,88.33,40703.23,2022-04-27 19:30:00,LUNA/USDT
...,...,...,...,...,...,...,...,...
94,1651249800000,87.30,87.44,85.84,86.13,82812.66,2022-04-29 16:30:00,LUNA/USDT
95,1651251600000,86.13,86.40,85.65,85.93,40838.28,2022-04-29 17:00:00,LUNA/USDT
96,1651253400000,85.92,86.17,85.32,85.74,34454.51,2022-04-29 17:30:00,LUNA/USDT
97,1651255200000,85.74,85.79,85.07,85.45,49690.84,2022-04-29 18:00:00,LUNA/USDT


## Executing the trade

In [5]:
## execute()

arrumar questão do currently_holding, criar stop loss, após atingir a alta 

In [6]:
currently_holding = False
while 1:
    # STEP 1: FETCH THE DATA
    ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
    if ticker_data is not None:
        # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
        trade_rec_type = get_trade_recommendation_MACD(ticker_data)
        print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")} TRADING RECOMMENDATION: {trade_rec_type}')

        # STEP 3: EXECUTE THE TRADE
        if (trade_rec_type == 'BUY' and not currently_holding):
            print(f'Placing {trade_rec_type} order')
            trade_successful, holding_quantity = execute_trade(binance_client, trade_rec_type, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY, holding_quantity)
            currently_holding = not currently_holding if trade_successful else currently_holding
            buy_position = True
            while buy_position:
                ticker_data = fetch_data(exchange, CCXT_TICKER_NAME, CANDLE_DURATION_IN_MIN)
                confirm_sell = confirm_sell_operation(ticker_data, 1.1)
                print(confirm_sell)
                if confirm_sell == "SELL":
                    print(f'Placing SELL order')
                    trade_successful, holding_quantity = execute_trade(binance_client, confirm_sell, TRADING_TICKER_NAME, INVESTMENT_AMOUNT_DOLLARS, DECIMALS_QUANTITY, holding_quantity)
                    currently_holding = not currently_holding if trade_successful else currently_holding
                    buy_position = not buy_position
                
                time.sleep(30)


        time.sleep(CANDLE_DURATION_IN_MIN*60)
    else:
        print(f'Unable to fetch ticker data - {CCXT_TICKER_NAME}. Retrying!!')
        time.sleep(5)


29/04/2022 16:01:48 TRADING RECOMMENDATION: WAIT
29/04/2022 16:31:49 TRADING RECOMMENDATION: WAIT
